In [1]:
!pip install numpy scipy gensim nltk

import warnings
warnings.filterwarnings('ignore')

import numpy as np

    100% |████████████████████████████████| 23.6MB 1.5MB/s 
    100% |████████████████████████████████| 1.4MB 14.4MB/s 
    100% |████████████████████████████████| 133kB 28.6MB/s 
    100% |████████████████████████████████| 61kB 22.7MB/s 
    100% |████████████████████████████████| 5.0MB 6.1MB/s 
    100% |████████████████████████████████| 552kB 26.7MB/s 
  Running setup.py bdist_wheel for smart-open ... - \ done
  Stored in directory: /root/.cache/pip/wheels/23/00/44/e5b939f7a80c04e32297dbd6d96fa3065af89ecf57e2b5f89f
  Running setup.py bdist_wheel for bz2file ... - done
  Stored in directory: /root/.cache/pip/wheels/81/75/d6/e1317bf09bf1af5a30befc2a007869fa6e1f516b8f7c591cb9
Successfully built smart-open bz2file


In [2]:
import gensim
from gensim.models import word2vec
! wget http://mattmahoney.net/dc/text8.zip -P /tmp
! unzip /tmp/text8.zip -d /tmp/
! head -c10000000 /tmp/text8 > /tmp/text8_10mo
 



--2018-11-28 23:39:28--  http://mattmahoney.net/dc/text8.zip
Resolving mattmahoney.net (mattmahoney.net)... 67.195.197.75
Connecting to mattmahoney.net (mattmahoney.net)|67.195.197.75|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 31344016 (30M) [application/zip]
Saving to: ‘/tmp/text8.zip’

text8.zip           100%[===================>]  29.89M   837KB/s    in 37s     

2018-11-28 23:40:05 (826 KB/s) - ‘/tmp/text8.zip’ saved [31344016/31344016]

Archive:  /tmp/text8.zip
  inflating: /tmp/text8              


In [3]:
# lire les donnés que nous avons téléchargé
sentences = word2vec.Text8Corpus('/tmp/text8_10mo')
for s in sentences:
  print("a sentence",s)
  break

a sentence ['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including', 'the', 'diggers', 'of', 'the', 'english', 'revolution', 'and', 'the', 'sans', 'culottes', 'of', 'the', 'french', 'revolution', 'whilst', 'the', 'term', 'is', 'still', 'used', 'in', 'a', 'pejorative', 'way', 'to', 'describe', 'any', 'act', 'that', 'used', 'violent', 'means', 'to', 'destroy', 'the', 'organization', 'of', 'society', 'it', 'has', 'also', 'been', 'taken', 'up', 'as', 'a', 'positive', 'label', 'by', 'self', 'defined', 'anarchists', 'the', 'word', 'anarchism', 'is', 'derived', 'from', 'the', 'greek', 'without', 'archons', 'ruler', 'chief', 'king', 'anarchism', 'as', 'a', 'political', 'philosophy', 'is', 'the', 'belief', 'that', 'rulers', 'are', 'unnecessary', 'and', 'should', 'be', 'abolished', 'although', 'there', 'are', 'differing', 'interpretations', 'of', 'what', 'this', 'means', 'anarchism', 'also', 'refers', 'to', 're

In [4]:
# Apprendre un modèle de embeddings sur text8
params = {
    'alpha': 0.05,   # learning rate
    'size':  100,    # number of dimensions for the dense representations
    'window': 5,     # context window size
    'iter':   5,     # nb of iterations 
    'min_count': 5,  # to ignore very rare words
    'negative': 5    # we need negative examples, how many?
}

my_model = word2vec.Word2Vec(sentences, **params)
print(my_model)


Word2Vec(vocab=19501, size=100, alpha=0.05)


In [5]:
word = 'services'
most_similar = my_model.wv.most_similar(positive=[word], topn=5)
print('word',word,'most_similar: ', most_similar)

word = 'japanese'
most_similar = my_model.wv.most_similar(positive=[word], topn=5)
print('word',word,'most_similar: ', most_similar)

word = 'six'
most_similar = my_model.wv.most_similar(positive=[word], topn=5)
print('word',word,'most_similar: ', most_similar)

word = 'good'
most_similar = my_model.wv.most_similar(positive=[word], topn=5)
print('word',word,'most_similar: ', most_similar)


# Calcul de la similarité cosinus entre deux mots
# cos_sim( a,b) =  a*b/|a||b|
cos_sim = np.dot( my_model.wv['services'],my_model.wv['facilities'] )/(np.linalg.norm(my_model.wv['services'], ord=2)*np.linalg.norm(my_model.wv['facilities'], ord=2))
print('consine similarity between', 'services' , 'facilities', cos_sim)


word services most_similar:  [('facilities', 0.8118171691894531), ('domestic', 0.7811663150787354), ('transportation', 0.7805765867233276), ('transport', 0.767768383026123), ('management', 0.7606114745140076)]
word japanese most_similar:  [('korean', 0.6604584455490112), ('canadian', 0.6587786674499512), ('finnish', 0.6348313093185425), ('hong', 0.6333869099617004), ('kong', 0.6218381524085999)]
word six most_similar:  [('seven', 0.906335175037384), ('eight', 0.900296688079834), ('four', 0.8972123265266418), ('five', 0.8874413967132568), ('three', 0.848983645439148)]
word good most_similar:  [('everything', 0.714170515537262), ('truly', 0.6715937256813049), ('something', 0.6712619662284851), ('knowing', 0.6653276681900024), ('mind', 0.6606166958808899)]
consine similarity between services facilities 0.81181717


In [6]:
# Relations semantiques dans les embeddings
most_similar = my_model.wv.most_similar(positive=['woman', 'husband'], negative=['man'], topn=5)
print('most_similar: ', most_similar)


most_similar:  [('mother', 0.7706172466278076), ('wife', 0.761226236820221), ('marriage', 0.7342442870140076), ('daughter', 0.7292108535766602), ('widow', 0.7134670615196228)]


### How to select your parameters?

Les deux parametres les plus importants sont la taille de la fenêtre et le nombre minimal de occurrences

La taille de fenêtre a un impact direct sur ce qui est modelisé :

- Des petites fenetres donnent des embeddings très syntaxiques (similarity)

- Des fenetres plus grandes donnent des embeddings plus sémantiques (relatedness)


In [0]:
# Importance de la taille de fenetre
params = {
    'alpha': 0.05,   # learning rate
    'size':  100,    # number of dimensions for the dense representations
    'window': 15,     # context window size
    'iter':   5,     # nb of iterations 
    'min_count': 5,  # to ignore very rare words
    'negative': 5    # we need negative examples, how many?
}

largecontext_model = word2vec.Word2Vec(sentences, **params)


params = {
    'alpha': 0.05,   # learning rate
    'size':  100,    # number of dimensions for the dense representations
    'window': 2,     # context window size
    'iter':   5,     # nb of iterations 
    'min_count': 5,  # to ignore very rare words
    'negative': 5    # we need negative examples, how many?
}

smallcontext_model = word2vec.Word2Vec(sentences, **params)





In [8]:
word = 'food'
most_similar = largecontext_model.wv.most_similar(positive=[word], topn=5)
print('Using a large context... word',word,'most_similar: ', most_similar)

most_similar = smallcontext_model.wv.most_similar(positive=[word], topn=5)
print('Using a small context... word',word,'most_similar: ', most_similar)




word = 'english'
most_similar = largecontext_model.wv.most_similar(positive=[word], topn=5)
print('Using a large context... word',word,'most_similar: ', most_similar)

most_similar = smallcontext_model.wv.most_similar(positive=[word], topn=5)
print('Using a small context... word',word,'most_similar: ', most_similar)



Using a large context... word food most_similar:  [('fish', 0.7793723344802856), ('crops', 0.7699872851371765), ('plants', 0.7595791816711426), ('farming', 0.752181887626648), ('alpaca', 0.7495522499084473)]
Using a small context... word food most_similar:  [('toxic', 0.7621316313743591), ('products', 0.7609580755233765), ('treatments', 0.756449818611145), ('fuel', 0.745498538017273), ('nutritional', 0.7451557517051697)]
Using a large context... word english most_similar:  [('british', 0.7093088626861572), ('speakers', 0.6906236410140991), ('dutch', 0.6887288093566895), ('words', 0.6858994960784912), ('pronunciation', 0.6566762924194336)]
Using a small context... word english most_similar:  [('dutch', 0.7248741388320923), ('spanish', 0.6748340129852295), ('italian', 0.6709585189819336), ('irish', 0.6550021767616272), ('swedish', 0.6448354125022888)]


### Stabilité des Embeddings

L'information utile dans les embeddings est encodé dans les similarités entre les mots et non pas dans leur position dans l'espace

Plusieurs executions de W2V sur les mêmes données produissent des embeddings très similaires.

C'est-à-dire, les plus proche voisins de chaque mot ne changent presque pas.

Mais malheureusement, deux modèles d'embeddings ne partagent pas le même espace de representation

In [0]:
# Nous allons apprendre deux embeddings avec les même parametres
params = {
    'alpha': 0.05,   # learning rate
    'size':  100,    # number of dimensions for the dense representations
    'window': 5,     # context window size
    'iter':   5,     # nb of iterations 
    'min_count': 5,  # to ignore very rare words
    'negative': 5    # we need negative examples, how many?
}

model_run1 = word2vec.Word2Vec(sentences, **params)
model_run2 = word2vec.Word2Vec(sentences, **params)


In [10]:
# Nous devrions trouver a peu près les mêmes voisins pour tous les mots
word = 'france'
most_similar_run1 = model_run1.wv.most_similar(positive=[word], topn=10)
most_similar_run2 = model_run2.wv.most_similar(positive=[word], topn=10)
for r1,r2 in zip(most_similar_run1,most_similar_run2):
  print('run1',r1,'run2',r2)


run1 ('spain', 0.8304658532142639) run2 ('spain', 0.8454252481460571)
run1 ('hungary', 0.814877986907959) run2 ('portugal', 0.7898801565170288)
run1 ('belgium', 0.7645211219787598) run2 ('belgium', 0.7831913232803345)
run1 ('poland', 0.761873722076416) run2 ('denmark', 0.7704712748527527)
run1 ('portugal', 0.760675311088562) run2 ('italy', 0.7536481618881226)
run1 ('norway', 0.7283957600593567) run2 ('invaded', 0.7478989958763123)
run1 ('italy', 0.7253027558326721) run2 ('hungary', 0.7306735515594482)
run1 ('prussia', 0.7235832214355469) run2 ('germany', 0.7286123037338257)
run1 ('germany', 0.713594913482666) run2 ('persia', 0.7213312387466431)
run1 ('montenegro', 0.7125173807144165) run2 ('palestine', 0.7195584177970886)


In [11]:
# Les similarités sont les mêmes, mais malheuresement les coordonées sont très differents 

word = 'france'
print(model_run1.wv[word])
print(model_run2.wv[word])

# We can compute the cosine similarity across models
# If the models were compatible the similarity ~1 for every word.
cos_sim = np.dot( model_run1.wv[word],model_run2.wv[word] )/(np.linalg.norm(model_run1.wv[word], ord=2)*np.linalg.norm(model_run2.wv[word], ord=2))
print('consine similarity across models for word:', word , 'is', cos_sim)


[ 0.10412591  0.36910257  0.2128633  -0.94174075 -1.4775884   0.97123146
  0.12979501  0.08196694  0.48873767  1.0581298   0.89284575 -1.1122179
 -0.18167287  0.28151572  1.4481906  -0.06247498  0.6485793   0.9260131
  0.20242517  0.5354726  -2.5048044  -0.55736315 -1.2102294  -1.2215934
 -0.08036274 -0.4425488   0.38013706  0.9614653   0.14880641 -0.08126009
 -0.19089879 -0.15448648 -0.26466215 -1.0582122   0.5334997  -0.01049083
  0.4911148   0.84772515 -0.2606806  -0.20451662 -0.9364205  -0.44798934
  0.04770898 -0.8343631  -0.541762   -0.7216429   0.3400382  -0.9914706
  0.32680726  0.8227253   0.31393048  0.39497668  0.31330696  0.7283616
  0.08742259  0.04278935  0.28140303  1.7641841   0.13293174 -0.4461315
  0.4774536   0.11408231  1.4818673   0.6621697  -0.7947394  -0.013896
 -0.9602215   0.5074674   0.623445    0.17093663  0.12411517  0.34137106
  1.1539719  -0.35197583  0.4585186   0.85181946 -0.74419236  0.8073199
 -1.5308359   0.80242217  0.99449843 -0.06994001  0.9608386 

### Alignement des embeddings après apprentissage

Voici comment aligner des embeddigs dans un même espace.

Il y a plusieurs methodes. La plus simple consiste a utiliser un dictionnaire de mots de reference a aligner.

Pour construire une la transformation lineaire qui nous envoie d'un espace vers l'autre


In [12]:
# try different values of the number of pivots. The more words used during the alignment the better
nb_pivots = 500
wordlist = list(model_run1.wv.vocab.keys())[0:nb_pivots]
model_run1.init_sims()
model_run2.init_sims()


# Extraire le vocabulaire de chaque model
vocab_m1 = set(model_run1.wv.vocab.keys())
vocab_m2 = set(model_run2.wv.vocab.keys())

# Extraire le vocabulaire en commun avec la liste 
common_vocab = list(vocab_m1&vocab_m2&set(wordlist))

syn0norms = []
# pour chaque modèle...
for m in [model_run1,model_run2]:
	# Remplacer syn0norm avec un nouveau array uniquement avec le mots en commun
	indices = [m.wv.vocab[w].index for w in common_vocab]
	old_arr = m.wv.syn0norm
	new_arr = np.array([old_arr[index] for index in indices])
	syn0norms.append( new_arr )
  
base_vecs = syn0norms[0]
other_vecs = syn0norms[1]

# produit scalaire entre les vecteurs de chaque espace
m = other_vecs.T.dot(base_vecs) 

# SVD method 
u, _, v = np.linalg.svd(m)
ortho = u.dot(v) 

# Replacer les embeddings avec les projections
model_run2.wv.syn0norm = model_run2.wv.syn0 = (model_run2.wv.syn0norm).dot(ortho)


word = 'france'
print(model_run1.wv[word])
print(model_run2.wv[word])

# We can compute the cosine similarity across models
# If the models were compatible the similarity ~1 for every word.
cos_sim = np.dot( model_run1.wv[word],model_run2.wv[word] )/(np.linalg.norm(model_run1.wv[word], ord=2)*np.linalg.norm(model_run2.wv[word], ord=2))
print('consine similarity across models for word:', word , 'is', cos_sim)




[ 0.10412591  0.36910257  0.2128633  -0.94174075 -1.4775884   0.97123146
  0.12979501  0.08196694  0.48873767  1.0581298   0.89284575 -1.1122179
 -0.18167287  0.28151572  1.4481906  -0.06247498  0.6485793   0.9260131
  0.20242517  0.5354726  -2.5048044  -0.55736315 -1.2102294  -1.2215934
 -0.08036274 -0.4425488   0.38013706  0.9614653   0.14880641 -0.08126009
 -0.19089879 -0.15448648 -0.26466215 -1.0582122   0.5334997  -0.01049083
  0.4911148   0.84772515 -0.2606806  -0.20451662 -0.9364205  -0.44798934
  0.04770898 -0.8343631  -0.541762   -0.7216429   0.3400382  -0.9914706
  0.32680726  0.8227253   0.31393048  0.39497668  0.31330696  0.7283616
  0.08742259  0.04278935  0.28140303  1.7641841   0.13293174 -0.4461315
  0.4774536   0.11408231  1.4818673   0.6621697  -0.7947394  -0.013896
 -0.9602215   0.5074674   0.623445    0.17093663  0.12411517  0.34137106
  1.1539719  -0.35197583  0.4585186   0.85181946 -0.74419236  0.8073199
 -1.5308359   0.80242217  0.99449843 -0.06994001  0.9608386 

### Mots hors vocabulaire

Je peux apprendre un embedding pour les mots inconnues avec word2vec. 

Par exemple, je peux reemplace tous les mots rares (count<5) par un token "_oov_"  et apprendre mes embeddings normalement

In [13]:
from collections import Counter

count = Counter([x for sent in sentences for x in sent])
oov_sentences = []
for sent in sentences:
  oov_sentences.append( [ s if(count[s]>5) else '_oov_' for s in sent ] )

  
for oov_sent,sent in zip(list(oov_sentences)[0:5], list(sentences)[0:5]):
  print(oov_sent)
  print(sent)
  

params = {
    'alpha': 0.05,   # learning rate
    'size':  100,    # number of dimensions for the dense representations
    'window': 15,     # context window size
    'iter':   5,     # nb of iterations 
    'min_count': 5,  # to ignore very rare words
    'negative': 5    # we need negative examples, how many?
}

oov_model = word2vec.Word2Vec(oov_sentences, **params)
print( 'oov_embedding', oov_model.wv['_oov_'] )

['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including', 'the', '_oov_', 'of', 'the', 'english', 'revolution', 'and', 'the', 'sans', '_oov_', 'of', 'the', 'french', 'revolution', 'whilst', 'the', 'term', 'is', 'still', 'used', 'in', 'a', 'pejorative', 'way', 'to', 'describe', 'any', 'act', 'that', 'used', 'violent', 'means', 'to', 'destroy', 'the', 'organization', 'of', 'society', 'it', 'has', 'also', 'been', 'taken', 'up', 'as', 'a', 'positive', 'label', 'by', 'self', 'defined', 'anarchists', 'the', 'word', 'anarchism', 'is', 'derived', 'from', 'the', 'greek', 'without', '_oov_', 'ruler', 'chief', 'king', 'anarchism', 'as', 'a', 'political', 'philosophy', 'is', 'the', 'belief', 'that', 'rulers', 'are', 'unnecessary', 'and', 'should', 'be', 'abolished', 'although', 'there', 'are', 'differing', 'interpretations', 'of', 'what', 'this', 'means', 'anarchism', 'also', 'refers', 'to', 'related', 'social', 